## Insight for the medical dataset

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from tqdm import tqdm
import pandas as pd
%matplotlib inline

### Paths

In [2]:
DATA_PATH = './data/'
BINDINGDB = 'BindingDB_All.tsv'
DRUGBANK = 'parsed_DrugBank.csv'
output_file = 'intermediate_results.csv'

### Loading the data

In [3]:
def load_BindingDB(file):
    file_path = DATA_PATH + file

    ## load the first row to get the number of columns
    df_first_row = pd.read_csv(file_path, sep='\t', nrows=1)

    ## number of columns to use
    df = pd.read_csv(file_path, sep='\t', header=0, usecols=range(df_first_row.shape[1]))
    return df

In [4]:
df = load_BindingDB(BINDINGDB)

C:\Users\sebge\AppData\Local\Temp\ipykernel_29384\2661906853.py:8: DtypeWarning: Columns (8,9,10,11,12,13,15,17,18,20,21,22,23,27,28,32,33,35,36,45,46,47,48,50,51,52,53,54,55,57,58,59,60,62,63,64,65,66,67,69,70,71,72,74,75,76,77,78,79,81,82,83,86,87,88,89,90,91,93,94,95,98,99,100,101,102,103,105,106,107,110,111,112,113,114,115,117,118,119,122,123,124,125,126,127,129,130,131,134,135,136,137,138,139,141,142,143,146,147,148,149,150,151,153,154,155,158,159,160,161,162,163,165,166,167,170,171,172,173,174,175,177,178,179,182,183,184,185,186,187,189,190,191) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t', header=0, usecols=range(df_first_row.shape[1]))


In [5]:
print(f"Number of rows: {len(df)}")
print(f"Number of cols: {len(df.columns)}")

Number of rows: 2927609
Number of cols: 194


In [6]:
import re
def keep_just_numeric(value):
    if type(value) != str:
        return -1
    
    cleaned_val = re.sub(r'[^\d.]+','', str(value)) ## There are random float / str in the dataset?? -> convert to str
    if(cleaned_val == ''): #It didn't contain any number?
        return -1
    return float(cleaned_val)

affinity_cols = ["Ki (nM)", "Kd (nM)"] ## Measures of binding affinity
ec_ic = ["EC50 (nM)", "IC50 (nM)"] #### Measures of inhibitory (IC50) and effective concentrations (EC50)
bind_unbind = ["kon (M-1-s-1)", "koff (s-1)"] ## Rates of binding / unbinding of ligands

binding_ligand_efficency_cols = affinity_cols + ec_ic + bind_unbind
filtered_df = df.copy()

for af_col in binding_ligand_efficency_cols:
    filtered_df[af_col] = filtered_df[af_col].apply(keep_just_numeric)
filtered_df.head()
    

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Recommended Name of Target Chain.12,UniProt (SwissProt) Entry Name of Target Chain.12,UniProt (SwissProt) Primary ID of Target Chain.12,UniProt (SwissProt) Secondary ID(s) of Target Chain.12,UniProt (SwissProt) Alternative ID(s) of Target Chain.12,UniProt (TrEMBL) Submitted Name of Target Chain.12,UniProt (TrEMBL) Entry Name of Target Chain.12,UniProt (TrEMBL) Primary ID of Target Chain.12,UniProt (TrEMBL) Secondary ID(s) of Target Chain.12,UniProt (TrEMBL) Alternative ID(s) of Target Chain.12
0,1,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CCCCCC(O)=...,InChI=1S/C31H42N2O7/c34-27(35)17-9-3-11-19-32-...,XGEGDSLAQZJGCW-HHGOQMMWSA-N,608734,"6-[(4R,5S,6S,7R)-4,7-dibenzyl-3-(5-carboxypent...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.24,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,InChI=1S/C31H34N6O3/c38-29-27(17-23-9-3-1-4-10...,UZLMEAPBHYEHAC-UNTBESQGSA-N,22,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.25,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,InChI=1S/C29H34N4O3/c34-27-25(16-21-8-3-1-4-9-...,HYNYUFZPPJMPOB-UTWJFGBXSA-N,23,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.41,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,InChI=1S/C29H40N2O4/c32-18-10-2-1-9-17-30-25(1...,YXVAZXDWVZTGGD-VIJSPRBVSA-N,24,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.80,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,InChI=1S/C28H38N2O4/c31-17-9-3-8-16-29-24(18-2...,WWTSWTPNILRSJX-XDZXDJIYSA-N,25,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.99,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Link to Drug Bank

In [7]:
drugbank = pd.read_csv(DATA_PATH + DRUGBANK, encoding='utf-8')
drugbank.head()

,id,name,unii,toxicity,class_kingdom,class_superclass,synonyms,categories,patent_approved,interaction,pubchem,chembl,SMILES,InChI,chebi,bindingdb
0,DB00001,Lepirudin,Y43GF64R34,The acute toxicity of intravenous lepirudin wa...,Organic Compounds,Organic Acids,"[Leu1, Thr2]-63-desulfohirudin|Desulfatohirudi...","Amino Acids, Peptides, and Proteins|Anticoagul...",1993-01-19,DB06605|DB06695|DB01254|DB01609|DB01586|DB0212...,46507011.0,CHEMBL1201666,NaN,NaN,NaN,NaN
1,DB00002,Cetuximab,PQX0D8J21J,The intravenous LD<sub>50</sub> is > 300 mg/kg...,Organic Compounds,Organic Acids,Cetuximab|Cétuximab|Cetuximabum,"Amino Acids, Peptides, and Proteins|Antibodies...",1999-03-02,DB00255|DB00269|DB00286|DB00655|DB00783|DB0089...,46507042.0,CHEMBL1201577,NaN,NaN,NaN,NaN
2,DB00003,Dornase alfa,953A26OA1Y,Adverse reactions occur at a frequency of < 1/...,Organic Compounds,Organic Acids,Deoxyribonuclease (human clone 18-1 protein mo...,"Amino Acids, Peptides, and Proteins|Cough and ...",2005-02-22|2004-10-26,NaN,46507792.0,CHEMBL1201431,NaN,NaN,NaN,NaN
3,DB00004,Denileukin diftitox,25E79B5CTM,NaN,Organic Compounds,Organic Acids,Denileukin|Denileukin diftitox|Interleukin-2/d...,"ADP Ribose Transferases|Amino Acids, Peptides,...",NaN,DB00012|DB00016|DB08894|DB09107|DB00281|DB0029...,46506950.0,CHEMBL1201550,NaN,NaN,NaN,NaN
4,DB00005,Etanercept,OP401G7OJC,NaN,Organic Compounds,Organic Acids,Etanercept|etanercept-szzs|etanercept-ykro|Rec...,"Agents reducing cytokine levels|Amino Acids, P...",2009-06-16|2000-03-14|2007-10-02,DB08879|DB00531|DB06643|DB00065|DB00008|DB0001...,46506732.0,CHEMBL1201572,NaN,NaN,NaN,NaN


In [8]:
len(filtered_df['DrugBank ID of Ligand'].dropna())

87465

In [9]:
smile_merged = pd.merge(filtered_df, drugbank, left_on='Ligand SMILES', right_on='SMILES')
len(smile_merged)

6191

In [10]:
# Rename columns in BindingDB to unify naming conventions
BindingDB = filtered_df.rename(columns={
    'PubChem CID': 'PubChem_CID',
    'ChEBI ID of Ligand': 'ChEBI_ID',
    'ChEMBL ID of Ligand': 'ChEMBL_ID',
    'DrugBank ID of Ligand': 'DrugBank_ID',
    'KEGG ID of Ligand': 'KEGG_ID',
    'ZINC ID of Ligand': 'ZINC_ID',
    'Ligand SMILES': 'SMILES',
    'Ligand InChI Key': 'InChI_Key',
    'BindingDB MonomerID': 'BindingDB_ID',
})

# Rename columns in DrugBank to unify naming conventions
DrugBank = drugbank.rename(columns={
    'chebi': 'ChEBI_ID',
    'chembl': 'ChEMBL_ID',
    'pubchem': 'PubChem_CID',
    'PubChem Substance': 'PubChem_SID',
    'id': 'DrugBank_ID',
    'bindingdb': 'BindingDB_ID',
    'ZINC': 'ZINC_ID',
    'SMILES': 'SMILES',
    'InChI': 'InChI_Key',
    'KEGG Compound': 'KEGG_ID'
})

BindingDB['Unique_ID'] = np.arange(len(BindingDB))
DrugBank['Unique_DrugBank_ID'] = np.arange(len(DrugBank))

In [11]:
import pandas as pd
import os
import gc

# Ensure the output file does not exist from previous runs
if os.path.exists(output_file):
    os.remove(output_file)

# List of identifiers to merge on
identifier_columns = [
    'PubChem_CID',
    'PubChem_SID',
    'ChEBI_ID',
    'ChEMBL_ID',
    'DrugBank_ID',
    'BindingDB_ID',
    'ZINC_ID',
    'SMILES',
    'InChI',
    'InChI_Key'
]

# Function to process and merge on each identifier individually
def merge_on_identifiers(df1, df2, identifiers, output_file):
    for identifier in identifiers:
        if identifier in df1.columns and identifier in df2.columns:
            print(f"Processing identifier: {identifier}")
            
            # Drop rows with NaN in the identifier columns
            df1_id = df1.dropna(subset=[identifier])
            df2_id = df2.dropna(subset=[identifier])
            
            # Convert identifier columns to string to avoid type mismatches
            if identifier in ['BindingDB_ID']:
                df1_id[identifier] = df1_id[identifier].astype(int)
                df2_id[identifier] = df2_id[identifier].astype(int)
            else:
                df1_id[identifier] = df1_id[identifier].astype(str)
                df2_id[identifier] = df2_id[identifier].astype(str)
            
            # Perform the merge
            merged_df = pd.merge(
                df1_id, df2_id, 
                on=identifier, 
                how='inner', 
                suffixes=('_BindingDB', '_DrugBank')
            )
            
            if not merged_df.empty:
                # Add a column to indicate which identifier was matched
                merged_df['Matched_On'] = identifier
                
                # Write to CSV in append mode
                merged_df.to_csv(
                    output_file, 
                    mode='a', 
                    index=False, 
                    header=not os.path.exists(output_file)
                )
            
            # Clean up memory
            del df1_id, df2_id, merged_df
            gc.collect()

# Perform the merge
merge_on_identifiers(BindingDB, DrugBank, identifier_columns, output_file)


Processing identifier: PubChem_CID


C:\Users\sebge\AppData\Local\Temp\ipykernel_29384\434813518.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_id[identifier] = df1_id[identifier].astype(str)
C:\Users\sebge\AppData\Local\Temp\ipykernel_29384\434813518.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_id[identifier] = df2_id[identifier].astype(str)


Processing identifier: ChEBI_ID


C:\Users\sebge\AppData\Local\Temp\ipykernel_29384\434813518.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_id[identifier] = df1_id[identifier].astype(str)
C:\Users\sebge\AppData\Local\Temp\ipykernel_29384\434813518.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_id[identifier] = df2_id[identifier].astype(str)


Processing identifier: ChEMBL_ID


...

C:\Users\sebge\AppData\Local\Temp\ipykernel_29384\434813518.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_id[identifier] = df2_id[identifier].astype(int)


Processing identifier: SMILES


C:\Users\sebge\AppData\Local\Temp\ipykernel_29384\434813518.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_id[identifier] = df2_id[identifier].astype(str)


Processing identifier: InChI_Key


C:\Users\sebge\AppData\Local\Temp\ipykernel_29384\434813518.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_id[identifier] = df1_id[identifier].astype(str)
C:\Users\sebge\AppData\Local\Temp\ipykernel_29384\434813518.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_id[identifier] = df2_id[identifier].astype(str)


In [12]:
merged_df = pd.read_csv(output_file)

C:\Users\sebge\AppData\Local\Temp\ipykernel_29384\3797673924.py:1: DtypeWarning: Columns (15,17,20,21,35,45,46,47,48,50,51,52,53,54,55,57,58,59,62,63,64,65,66,67,69,70,71,74,75,76,77,78,79,86,87,88,89,90,91,98,99,100,101,102,103,105,106,107,110,111,112,113,114,115,122,123,124,125,126,127,129,130,131,134,135,136,137,138,139,146,147,148,149,150,151,158,159,160,161,162,163,170,171,172,173,174,175,182,183,184,185,186,187,204,205,208) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv(output_file)


In [13]:
merged_df['Matched_On'].value_counts()

Matched_On
InChI_Key       147426
BindingDB_ID    138645
ChEMBL_ID        96639
DrugBank_ID      83430
ChEBI_ID         74904
SMILES            6191
PubChem_CID          9
Name: count, dtype: int64

In [14]:
len(merged_df)

547244

In [15]:
merged_df['Unique_ID'].nunique()

152520

In [16]:
merged_df.drop_duplicates(subset=['Unique_ID'], inplace=True)

In [17]:
merged_df['Unique_DrugBank_ID'].nunique()

5777

In [18]:
DrugBank['Unique_DrugBank_ID'].nunique()

16581

In [19]:
merged_df['Unique_DrugBank_ID'].nunique() / DrugBank['Unique_DrugBank_ID'].nunique()

0.3484108316748085

In [20]:
len(DrugBank)

16581

In [21]:
merged_df['Unique_DrugBank_ID'].nunique() / len(BindingDB) * 100

0.19732826343955087

In [22]:
BindingDB.loc[BindingDB['DrugBank_ID'].dropna().index][['DrugBank_ID', 'SMILES', 'BindingDB_ID']].head()

,DrugBank_ID,SMILES,BindingDB_ID
29,DB06910,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC=C)C(=O)...,152
131,DB06910,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC=C)C(=O)...,152
235,DB01264,CC(C)CN(C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)O[C@H]...,8125
246,DB01264,CC(C)CN(C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)O[C@H]...,8125
814,DB02033,Oc1c2CCCCCCc2oc(=O)c1[C@@H](C1CC1)c1cccc(NS(=O...,763


In [23]:
DrugBank.head(2)

,DrugBank_ID,name,unii,toxicity,class_kingdom,class_superclass,synonyms,categories,patent_approved,interaction,PubChem_CID,ChEMBL_ID,SMILES,InChI_Key,ChEBI_ID,BindingDB_ID,Unique_DrugBank_ID
0,DB00001,Lepirudin,Y43GF64R34,The acute toxicity of intravenous lepirudin wa...,Organic Compounds,Organic Acids,"[Leu1, Thr2]-63-desulfohirudin|Desulfatohirudi...","Amino Acids, Peptides, and Proteins|Anticoagul...",1993-01-19,DB06605|DB06695|DB01254|DB01609|DB01586|DB0212...,46507011.0,CHEMBL1201666,NaN,NaN,NaN,NaN,0
1,DB00002,Cetuximab,PQX0D8J21J,The intravenous LD<sub>50</sub> is > 300 mg/kg...,Organic Compounds,Organic Acids,Cetuximab|Cétuximab|Cetuximabum,"Amino Acids, Peptides, and Proteins|Antibodies...",1999-03-02,DB00255|DB00269|DB00286|DB00655|DB00783|DB0089...,46507042.0,CHEMBL1201577,NaN,NaN,NaN,NaN,1


In [24]:
# Left join binding db with merged_df (on Unique_ID), don't duplicate columns though
binding_readded = pd.merge(BindingDB, merged_df, on='Unique_ID', how='left', suffixes=('', '_y'))

In [25]:
binding_readded.head(2)

,BindingDB Reactant_set_id,SMILES,Ligand InChI,InChI_Key,BindingDB_ID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,categories,patent_approved,interaction,ChEMBL_ID_DrugBank,SMILES_DrugBank,InChI_Key_DrugBank,ChEBI_ID_DrugBank,BindingDB_ID_DrugBank,Unique_DrugBank_ID,Matched_On
0,1,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CCCCCC(O)=...,InChI=1S/C31H42N2O7/c34-27(35)17-9-3-11-19-32-...,XGEGDSLAQZJGCW-HHGOQMMWSA-N,608734,"6-[(4R,5S,6S,7R)-4,7-dibenzyl-3-(5-carboxypent...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.24,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,InChI=1S/C31H34N6O3/c38-29-27(17-23-9-3-1-4-10...,UZLMEAPBHYEHAC-UNTBESQGSA-N,22,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.25,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
all_cols = list(binding_readded.columns)
cols_to_keep = []

for col in all_cols:
    if not ('_y' in col and col.split('_y')[0] in list(BindingDB.columns)):
        cols_to_keep.append(col)

In [27]:
# Keep only the columns in cols_to_keep
cleaned_readded = binding_readded[cols_to_keep]

In [28]:
cleaned_readded

,BindingDB Reactant_set_id,SMILES,Ligand InChI,InChI_Key,BindingDB_ID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,categories,patent_approved,interaction,ChEMBL_ID_DrugBank,SMILES_DrugBank,InChI_Key_DrugBank,ChEBI_ID_DrugBank,BindingDB_ID_DrugBank,Unique_DrugBank_ID,Matched_On
0,1,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CCCCCC(O)=...,InChI=1S/C31H42N2O7/c34-27(35)17-9-3-11-19-32-...,XGEGDSLAQZJGCW-HHGOQMMWSA-N,608734,"6-[(4R,5S,6S,7R)-4,7-dibenzyl-3-(5-carboxypent...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.24,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,InChI=1S/C31H34N6O3/c38-29-27(17-23-9-3-1-4-10...,UZLMEAPBHYEHAC-UNTBESQGSA-N,22,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.25,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,InChI=1S/C29H34N4O3/c34-27-25(16-21-8-3-1-4-9-...,HYNYUFZPPJMPOB-UTWJFGBXSA-N,23,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.41,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,InChI=1S/C29H40N2O4/c32-18-10-2-1-9-17-30-25(1...,YXVAZXDWVZTGGD-VIJSPRBVSA-N,24,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.80,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,InChI=1S/C28H38N2O4/c31-17-9-3-8-16-29-24(18-2...,WWTSWTPNILRSJX-XDZXDJIYSA-N,25,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.99,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2927604,51531022,CC[C@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2ccc(...,InChI=1S/C24H26F3N3O2S/c1-3-19-14-29(13-17-9-1...,ZBKXSRSXIGZPCY-IBGZPJMESA-N,50615961,CHEMBL5268424,Peroxisome proliferator-activated receptor gamma,Mus musculus,-1.00,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927605,51531023,CC[C@@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2ccc...,InChI=1S/C24H26F3N3O2S/c1-3-19-14-29(13-17-9-1...,ZBKXSRSXIGZPCY-LJQANCHMSA-N,50615962,CHEMBL5272996,Peroxisome proliferator-activated receptor gamma,Mus musculus,-1.00,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927606,51531024,CC(C)[C@@H]1CN(Cc2cc(C)cc(CC(O)=O)c2)CCN1c1nc2...,InChI=1S/C25H28F3N3O2S/c1-15(2)21-14-30(13-18-...,RVZUBLMVVIWYLZ-NRFANRHFSA-N,50615964,CHEMBL5277165,Peroxisome proliferator-activated receptor gamma,Mus musculus,-1.00,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2927607,51531025,COc1ccc(cc1)N(C)c1nc(C)nc2[nH]ccc12,InChI=1S/C15H16N4O/c1-10-17-14-13(8-9-16-14)15...,HKPVMULFKBBABY-UHFFFAOYSA-N,50615967,CHEMBL1275943,Similar to alpha-tubulin isoform 1,NaN,-1.00,2600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Add Protein Data from DrugBank

In [29]:
DrugBankProteins = pd.read_csv('data/parsed_proteins_DrugBank.csv')
DrugBankProteins.rename(columns={'id': 'drugbank_swissprot_id'}, inplace=True)

In [30]:
complete_merged_df = pd.merge(cleaned_readded, DrugBankProteins, left_on='UniProt (SwissProt) Primary ID of Target Chain', right_on='drugbank_swissprot_id', how='left')

In [31]:
complete_merged_df['drugbank_swissprot_id'].notna().sum() / len(complete_merged_df)

0.7263866178851069

In [32]:
complete_merged_df.to_csv('Merged Binding DrugBank with Proteins.csv', index=False)

In [76]:
import os
if os.path.exists(output_file):
    os.remove(output_file)